# Tarefas de Classificação


## Estudo de caso: dados sobre o censo de 1994 dos EUA
Este dataset contém dados coletados pelo censo norte americano de 1994, contendo atributos sobre cada pessoa intrevistada, sendo que o último deles indica se a pessoa possuia renda maior ou menor do que $50.000,00 anuais.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

df = pd.read_csv('adult.csv')
df.head(10)

FileNotFoundError: [Errno 2] File adult.csv does not exist: 'adult.csv'

### Separando os conjuntos de treino e teste

In [ ]:
from sklearn.model_selection import train_test_split

# separando os conjuntos de dados de treino e teste
df_treino, df_teste = train_test_split(df, test_size=0.2, random_state=42)

# separando a coluna alvo do conjunto de treino
df_treino_labels = df_treino['class'].copy()
df_treino        = df_treino.drop(columns='class')

# separando a coluna alvo do conjunto de teste
df_teste_labels = df_teste['class'].copy()
df_teste        = df_teste.drop(columns='class')

### Pré-processamento

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Atributos escolhidos:
nomes_atributos_numericos   = ['age','fnlwgt','education-num','capital-gain',
                               'capital-loss','hours-per-week']
nomes_atributos_categoricos = ['workclass','education','occupation','relationship',
                               'marital-status','race','sex','native-country']

pipeline_atr_numericos = Pipeline([
    ('imputer',SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
])

preproc_completo = ColumnTransformer([
    ('numericos',   pipeline_atr_numericos, nomes_atributos_numericos),
    ('categoricos', OneHotEncoder(),        nomes_atributos_categoricos),
    ], 
    sparse_threshold=0)

# pre-processamento do conjunto de treino
X_treino = preproc_completo.fit_transform(df_treino)

# pre-processamento do conjunto de teste
X_teste = preproc_completo.transform(df_teste)

In [ ]:
# Coluna alvo para os conjuntos de treino e teste
y_treino = df_treino_labels.values
y_teste = df_teste_labels.values

In [ ]:
y_treino

In [ ]:
X_treino.shape

In [ ]:
y_treino.shape

In [ ]:
X_teste.shape

In [ ]:
y_teste.shape

## Regressão Logística com SKLearn

In [ ]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(max_iter=300)

log_reg.fit(X_treino, y_treino)
log_reg.classes_

### Probabilidade estimada pela regressão logística

In [ ]:
y_probabilidade_reglog = log_reg.predict_proba(X_teste)
np.round(y_probabilidade_reglog,decimals=2)

### Previsões pela regressão logística

In [ ]:
y_previsto_reglog = log_reg.predict(X_teste)
y_previsto_reglog

In [ ]:
y_teste

### Acurácia pela regressão logística

In [ ]:
y_teste, y_teste.shape

In [ ]:
acertos_reglog = y_previsto_reglog == y_teste
acertos_reglog

In [ ]:
valores_reglog, quantidades_reglog = np.unique(acertos_reglog, return_counts=True)
print(np.asarray((valores_reglog, quantidades_reglog)))

In [ ]:
print("Acurácia: ", quantidades_reglog[1]/y_teste.shape[0])

## Árvore de Decisão  com SKLearn

In [ ]:
from sklearn.tree import DecisionTreeClassifier

arvore = DecisionTreeClassifier(max_depth=5)
arvore.fit(X_treino, y_treino)
arvore.classes_

### Probabilidade estimada pela árvore de decisão

In [ ]:
y_probabilidade_arvore = arvore.predict_proba(X_teste)
np.round(y_probabilidade_arvore,decimals=2)

### Previsões pela árvore de decisão

In [ ]:
y_previsto_arvore = arvore.predict(X_teste)
y_previsto_arvore

In [ ]:
acertos_arvore = y_previsto_arvore == y_teste
acertos_arvore

In [ ]:
valores_arvore, quantidades_arvore = np.unique(acertos_arvore, return_counts=True)
print(np.asarray((valores_arvore, quantidades_arvore)))

### Acurácia pela árvore de decisão

In [ ]:
from sklearn.metrics import accuracy_score

print("Acurácia: ", accuracy_score(y_teste,y_previsto_arvore))

### Visualização da árvore de decisão

In [ ]:
nomes_atributos = np.append( nomes_atributos_numericos, 
                             preproc_completo.named_transformers_['categoricos'].get_feature_names())

In [ ]:
from sklearn import tree

plt.figure(figsize=(24,12))

tree.plot_tree(arvore,
               feature_names=nomes_atributos,
               class_names=arvore.classes_,
               rounded=True,
               filled=True,
               proportion=True,
               fontsize=16)

## Árvore de decisão sem pré-processamento dos atributos numéricos
Eliminaremos o pre-processamento dos atributos numéricos, mas manteremos o pré-processsamento dos atributos categóricos. A implementação do SKLearn não suporta atributos categóricos. 

In [ ]:
preproc_completo_2 = ColumnTransformer([
    ('numericos',   'passthrough',    nomes_atributos_numericos),
    ('categoricos', OneHotEncoder(),  nomes_atributos_categoricos),
    ], 
    sparse_threshold=0)

# pre-processamento do conjunto de treino
X_treino_tree = preproc_completo_2.fit_transform(df_treino)

# pre-processamento do conjunto de teste
X_teste_tree = preproc_completo_2.transform(df_teste)

In [ ]:
arvore_2 = DecisionTreeClassifier(max_depth=2)
arvore_2.fit(X_treino_tree, y_treino)
arvore_2.classes_

In [ ]:
y_probabilidade_arvore_2 = arvore_2.predict_proba(X_teste_tree)
np.round(y_probabilidade_arvore_2,decimals=2)

In [ ]:
y_previsto_arvore_2 = arvore_2.predict(X_teste_tree)
y_previsto_arvore_2

In [ ]:
acertos_arvore_2 = y_previsto_arvore_2 == y_teste
acertos_arvore_2

In [ ]:
valores_arvore_2, quantidades_arvore_2 = np.unique(acertos_arvore_2, return_counts=True)
print(np.asarray((valores_arvore_2, quantidades_arvore_2)))

In [ ]:
print("Acurácia: ", accuracy_score(y_teste,y_previsto_arvore_2))

In [ ]:
plt.figure(figsize=(24,12))
tree.plot_tree(arvore_2,
               feature_names=nomes_atributos,
               class_names=arvore_2.classes_,
               rounded=True,
               filled=True,
               proportion=True,
               fontsize=16)